In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
df = pd.DataFrame(pd.read_csv("../assets/normalized_pyramid_data.csv"))

In [ ]:
key = ['unknown', 'pyramid?']
complexes = df[~df['pyramid_complex'].isin(key)]
kings_and_queens = complexes.groupby(['pyramid_complex', 'royal_status'], sort=False).size().reset_index(name='count')

confirmed_count_fig = px.bar(kings_and_queens, 
             x='pyramid_complex', 
             y='count', 
             color='royal_status',
             title='Number of Confirmed Pyramids At Each Complex',
             labels={
                 'pyramid_complex': 'Pyramid Complex',
                 'count': 'Number of Pyramids',
                 'royal_status': 'Royal Status'
             },
             custom_data=['royal_status'])
confirmed_count_fig.update_traces(
    hovertemplate='<br>'.join([
        'Royal status: %{customdata[0]}',
        'Pyramid Complex: %{x}',
        'Total: %{y}',
        '<extra></extra>'
    ])
)
confirmed_count_fig.show()
df

In [ ]:
dynasty_pyramids = df.groupby(['dynasty', 'royal_status'], sort=False).size().reset_index(name='count')
dynasty_count_fig = px.bar(dynasty_pyramids, 
             x='dynasty', 
             y='count', 
             color='royal_status',
             title='Number of Pyramids Per Dynasty',
             labels={
                 'dynasty': 'Dynasty',
                 'count': 'Number of Pyramids (Including Speculations)',
                 'royal_status': 'Royal Status'
             },
             custom_data=['royal_status'])
dynasty_count_fig.update_traces(
    hovertemplate='<br>'.join([
        'Royal status: %{customdata[0]}',
        'Dynasty: %{x}',
        'Total: %{y}',
        '<extra></extra>'
    ])
)
dynasty_count_fig.show()

In [ ]:
# Attempting to recreate George's suggested visualization (picture on my phone)

# Trying to fill 'start_of_reign' fully so that sorts work correctly
# TODO: Add this functionality to the cleanup script
unique_comp = complexes['pyramid_complex'].unique()
temp = df

for comp in unique_comp:
    start = temp[temp['pyramid_complex'] == comp]['start_of_reign'].max()
    temp[temp['pyramid_complex'] == comp]['start_of_reign'] = temp[temp['pyramid_complex'] == comp]['start_of_reign'].replace(np.nan, start)

temp.loc[temp['pyramid_complex'] == 'Sneferu 3', 'start_of_reign'] = 2574   # This had to be done to get it in the correct order (value was missing)
temp.dropna(subset='height', inplace=True)
temp.sort_values(by='start_of_reign', ascending=False, inplace=True)

# Getting the height column to be numeric
def average_of_two(val):
    if isinstance(val, int) or isinstance(val, float) or pd.isna(val): return val

    if ',' in val: return 72    # Temporary: Deals with that one weird value

    nums = val.split('-')
    if len(nums) == 1: return float(nums[0])
    return (float(nums[0]) + float(nums[1])) / 2

temp['height'] = temp['height'].map(average_of_two).astype(float)
tl = temp[['pyramid_complex', 'pyramid_owner', 'start_of_reign', 'end_of_reign', 'length_of_reign', 'height', 'royal_status', 'relationship_to_king']]

# Plotly stuff
grouped_heights_fig = px.bar(tl, 
             x='pyramid_complex', 
             y='height', 
             color='royal_status',
             title='Height of Pyramids At Each Complex By Status',
             labels={
                 'pyramid_complex': 'Pyramid Complex',
                 'height': 'Height (meters)',
                 'royal_status': 'Royal Status'
             },
             barmode='group',
             custom_data=['royal_status', 'pyramid_owner', tl['relationship_to_king'].fillna('Self')])
grouped_heights_fig.update_traces(
    hovertemplate='<br>'.join([
        'Royal status: %{customdata[0]}',
        'Pyramid Complex: %{x}',
        'Height: %{y} m',
        'Pyramid Owner: %{customdata[1]}',
        'Relationship To King: %{customdata[2]}',
        '<extra></extra>'
    ])
)
grouped_heights_fig.show()

In [ ]:
complex_heights = tl.groupby('pyramid_complex', sort=False, as_index=False)['height'].sum()

line_heights_fig = px.line(
    complex_heights,
    x='pyramid_complex',
    y='height',
    title='Total Height of All Pyramids At Each Complex',
    labels={
        'pyramid_complex': 'Pyramid Complex',
        'height': 'Aggregated Height (meters)'
    }
)
line_heights_fig.update_traces(
    hovertemplate='<br>'.join([
        'Pyramid Complex: %{x}',
        'Aggregated Height: %{y} m'
    ])
)
line_heights_fig.show()

In [ ]:
# Exporting to HTML
from jinja2 import Template

output_html_path = r'../pages/index.html'
input_template_path = r'../templates/template.html'

plotly_jinja_data = {
    'grouped_heights':grouped_heights_fig.to_html(full_html=False),
    'dynasty_count':dynasty_count_fig.to_html(full_html=False),
    'confirmed_count':confirmed_count_fig.to_html(full_html=False),
    'line_heights':line_heights_fig.to_html(full_html=False)
}

with open(output_html_path, 'w', encoding='utf-8') as out:
    with open(input_template_path) as template:
        j2_template = Template(template.read())
        out.write(j2_template.render(plotly_jinja_data))

In [ ]:
import plotly.graph_objects as go

# Get the groups with relevant data
com_heights = tl.groupby(['pyramid_complex', 'start_of_reign', 'end_of_reign', 'length_of_reign'], sort=False, as_index=False)['height'].sum()
com_heights['start_of_reign'] = -com_heights['start_of_reign']
com_heights['end_of_reign'] = -com_heights['end_of_reign']

# Select random colors for each complex
# String below copied from the output of an error, which listed valid CSS colors
colors = 'aliceblue, antiquewhite, aqua, aquamarine, azure, beige, bisque, black, blanchedalmond, blue, blueviolet, brown, burlywood, cadetblue, chartreuse, chocolate, coral, cornflowerblue, cornsilk, crimson, cyan, darkblue, darkcyan, darkgoldenrod, darkgray, darkgrey, darkgreen, darkkhaki, darkmagenta, darkolivegreen, darkorange, darkorchid, darkred, darksalmon, darkseagreen, darkslateblue, darkslategray, darkslategrey, darkturquoise, darkviolet, deeppink, deepskyblue, dimgray, dimgrey, dodgerblue, firebrick, floralwhite, forestgreen, fuchsia, gainsboro, ghostwhite, gold, goldenrod, gray, grey, green, greenyellow, honeydew, hotpink, indianred, indigo, ivory, khaki, lavender, lavenderblush, lawngreen, lemonchiffon, lightblue, lightcoral, lightcyan, lightgoldenrodyellow, lightgray, lightgrey, lightgreen, lightpink, lightsalmon, lightseagreen, lightskyblue, lightslategray, lightslategrey, lightsteelblue, lightyellow, lime, limegreen, linen, magenta, maroon, mediumaquamarine, mediumblue, mediumorchid, mediumpurple, mediumseagreen, mediumslateblue, mediumspringgreen, mediumturquoise, mediumvioletred, midnightblue, mintcream, mistyrose, moccasin, navajowhite, navy, oldlace, olive, olivedrab, orange, orangered, orchid, palegoldenrod, palegreen, paleturquoise, palevioletred, papayawhip, peachpuff, peru, pink, plum, powderblue, purple, red, rosybrown, royalblue, rebeccapurple, saddlebrown, salmon, sandybrown, seagreen, seashell, sienna, silver, skyblue, slateblue, slategray, slategrey, snow, springgreen, steelblue, tan, teal, thistle, tomato, turquoise, violet, wheat, white, whitesmoke, yellow, yellowgreen'.split(', ')
choices = np.random.default_rng().choice(len(colors), len(com_heights), replace=False)

# Create figure
figure = go.Figure()
'''
# This adds the bars in a single trace, which doesn't work well with the legend, which
# expects a trace for each bar.
figure.add_trace(
    go.Bar(
        x=com_heights['start_of_reign'],
        y=com_heights['height'],
        width=com_heights['length_of_reign'],
        text=com_heights['pyramid_complex'],
        offset=0.5,
        marker=dict(
            color=choices,
            colorscale=colors
        )
    )
)'''

# Add each bar individually as a trace
for complex, start, end, reign, height in com_heights.values:
    figure.add_trace(
        go.Bar(
            x=[start],
            y=[height],
            width=[reign],
            text=[complex],
            name=complex,
            offset=0.5,
            customdata=[complex, start, end, reign, height],
            hovertemplate='<br>'.join([
                'Pyramid Complex: %{customdata[0]}',
                'Reign: %{customdata[1]} - %{customdata[2]} BCE',
                'Length of Reign: %{customdata[3]} Years',
                'Agg. Height: %{customdata[4]} m',
                '<extra></extra>'
            ])
        )
    )
figure.update_xaxes(
    title_text='Time (Years BCE)',
    minor={'showgrid': True, 'dtick': 10, 'tick0': 0.1})
figure.update_yaxes(title_text='Aggregated Height (Meters)')
figure.update_layout(
    autosize=False,
    width=1000,
    height=800,
    title={'text': 'Total Height of All Pyramids At Each Complex'},
    xaxis={
        'tickvals': [i for i in range(-2700, -2100, 25)],
        'tickmode': 'array',
        'showticklabels': True,
        'ticks': 'outside',
        'ticklen': 5
    }
)
figure.show()

# NOTE: Sneferu 3 missing due to absence of start and end values (I think), and 
# not included after the fact because the bar would overlap with Sneferu 2. Figure 
# out a way to resolve this issue (combine all Sneferus?)

# NOTE: They want to see Ibi on this timeline, however his start and end are not listed, so
# he cannot be placed anywhere

# NOTE: Nebka also missing because there is no height listed for his pyramid

In [ ]:
queens = temp[temp['royal_status'] == 'Queen']
queen_data = queens[['pyramid_owner', 'dynasty', 'royal_status', 'daughter_of', 'royal_mother_title', 'likely_wife', 'wife_title', 'vizier', 'regent', 'relationship_to_king', 'height']]
queen_data['dynasty'] = queen_data['dynasty'].astype(int)

by_dyn = queen_data.groupby('dynasty', as_index=False)

'''fig_3d = px.scatter_3d(
    queen_data,
    x='vizier',
    y='likely_wife',
    z='regent',
    color='royal_mother_title'
)'''

# FACET is the word you keep looking for

vizier_dyn = queen_data.groupby('dynasty', as_index=False)['vizier'].value_counts()
fig = px.bar(
    vizier_dyn,
    x='dynasty',
    y='count',
    color='vizier',
    color_discrete_sequence=['red', 'blue'],
    barmode='stack',
    labels={
        'dynasty': 'Dynasty',
        'count': 'Total',
        'vizier': 'Vizier?'
    },
    title='Vizier Queen Pyramids By Dynasty'
)

fig.show()


In [ ]:
royal_mother_title_dyn = queen_data.groupby('dynasty', as_index=False)['royal_mother_title'].value_counts()
fig = px.bar(
    royal_mother_title_dyn,
    x='dynasty',
    y='count',
    color='royal_mother_title',
    color_discrete_sequence=['red', 'blue'],
    barmode='stack',
    labels={
        'dynasty': 'Dynasty',
        'count': 'Total',
        'royal_mother_title': 'Royal Mother?'
    },
    title='Royal Mother Queen Pyramids By Dynasty'
)
fig.show()

In [ ]:
likely_wife_dyn = queen_data.groupby('dynasty', as_index=False)['likely_wife'].value_counts()
fig = px.bar(
    likely_wife_dyn,
    x='dynasty',
    y='count',
    color='likely_wife',
    barmode='stack',
    labels={
        'dynasty': 'Dynasty',
        'count': 'Total',
        'likely_wife': 'Likely Wife?'
    },
    title='Wife Queen Pyramids By Dynasty'
)
fig.show()



In [ ]:
regent_dyn = queen_data.groupby('dynasty', as_index=False)['regent'].value_counts()
fig = px.bar(
    regent_dyn,
    x='dynasty',
    y='count',
    color='regent',
    color_discrete_sequence=['red', 'blue'],
    barmode='stack',
    labels={
        'dynasty': 'Dynasty',
        'count': 'Total',
        'regent': 'Regent?'
    },
    title='Regent Queen Pyramids By Dynasty'
)

fig.show()

In [ ]:
# Suggestion from meeting:
# Queen scatterplot representing the above with shapes, colors, outlines of shapes, etc.

# NOTE: The 3D scatterplot probably isn't ideal for a paper, can be unclear where exactly
# certain points are positioned. Try encoding all of the information in a 2d representation.
fig = px.scatter_3d(
    queen_data,
    x='dynasty',
    y='height',
    z='royal_mother_title',
    symbol='likely_wife',
    color='regent',
    symbol_map={True: 'cross', False: 'square'}
)
fig.show()

In [ ]:
# Reshape queen data from wide to long (Binary categories get put into a new column, each category applied to a specific queen given a row, with the status of that category in another column)

melted_queens = queen_data.melt(ignore_index=False, id_vars=['dynasty', 'height', 'pyramid_owner', 'relationship_to_king', 'daughter_of'], value_vars=['vizier', 'regent', 'royal_mother_title', 'likely_wife', 'wife_title']).reset_index()

In [ ]:
# Strip scatter plot version 1

fig = px.strip(
    melted_queens,
    x='dynasty',
    y='height',
    color='variable',
    facet_col='value'
)
fig.show()

In [ ]:
# Strip catter plot version 2 (I think this is the way to go)

fig = px.strip(
    melted_queens,
    x='dynasty',
    y='height',
    color='value',
    color_discrete_sequence=['red', 'blue'],
    facet_col='variable',
    stripmode='overlay',
    labels={
        'dynasty': 'Dynasty',
        'height': 'Height of Pyramid (meters)',
        'variable': 'Status',
        'value': 'Has Status?'
    },
    title='Status of Queens with Pyramids By Dynasty',
    custom_data=['pyramid_owner', 'relationship_to_king', melted_queens['daughter_of'].fillna('NA'), 'height'],
)
fig.update_traces(
    hovertemplate='<br>'.join([
        'Queen: %{customdata[0]}',
        'Relationship to King: %{customdata[1]}',
        'Daughter of: %{customdata[2]}',
        'Pyramid Height: %{customdata[3]} m'
        '<extra></extra>'
    ]))
fig.show()